In [17]:
import os
import re
import PyPDF2
import nltk
from sentence_transformers import SentenceTransformer, util

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danibacaicoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danibacaicoa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'  
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\danibacaicoa\AppData\Local\Temp\ipykernel_16520\201937189.py:1: SyntaxWarning: invalid escape sequence '\M'
  folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'
c:\Users\danibacaicoa\Desktop\VS projects\Library_manager-1\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
folder

'H:\\Mi unidad\\Biblioteca\\Mathematics\\Analysis'

In [20]:
def extract_text_from_pdfs(pdf_folder):
    pdf_texts = {}
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page_num in range(len(reader.pages)):
                    text += reader.pages[page_num].extract_text()
                    
                pdf_texts[filename] = text
    return pdf_texts

In [21]:
Text = extract_text_from_pdfs(folder)


In [31]:
list(Text.keys())[0]

'[B] 2015 - Theory of Distributions (Georgiev, 2015).pdf'

In [23]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum()]
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = [word for word in tokens if word.lower() not in stopwords]
    return ' '.join(tokens)


In [33]:
first_book_title = list(Text.keys())[0]

preprocess_text(first_book_title)

'B 2015 Theory Distributions Georgiev 2015'

In [34]:
first_book = Text[first_book_title]

preprocess_text(first_book)

'Theory DistributionsTheory DistributionsSvetlin Georgiev Theory Distributions 123Svetlin Georgiev Department Differential EquationsUniversity Soﬁa St Kliment Ohridski Soﬁa Bulgaria ISBN ISBN eBook DOI Library Congress Control Number 2015944329 Springer Cham Heidelberg New York Dordrecht London Springer International P ublishing Switzerland 2015 work subject copyright rights rese rved Publisher whether whole part material concerned speciﬁcally rights tra nslation reprinting reuse illustrations recitation broadcasting reproduction microﬁlms oth er physical way transmission information storage retrieval electronic adaptation com puter software similar dissimilar methodology known hereafter use general descriptive names registered names trademarks service marks etc publication imply even absence speciﬁc statement names exempt relevant protective laws regulations therefore free general use publisher authors editors safe assume advice information book believed true accurate date pub licatio

In [37]:
def create_embeddings(texts):
    embeddings = {}
    for filename, text in texts.items():
        processed_text = preprocess_text(text)
        embeddings[filename] = embedding_model.encode(processed_text, convert_to_tensor=True)
    return embeddings

In [36]:
pdf_texts = extract_text_from_pdfs(folder)


NameError: name 'create_embeddings' is not defined

In [38]:
pdf_embeddings = create_embeddings(pdf_texts)

In [39]:
pdf_embeddings

{'[B] 2015 - Theory of Distributions (Georgiev, 2015).pdf': tensor([-6.1457e-02, -7.3564e-02, -4.7856e-02, -5.0248e-02, -2.4108e-02,
          3.0789e-02, -6.4676e-03,  5.4196e-02,  2.4668e-02,  3.2120e-02,
          5.6907e-02,  2.7475e-03,  5.6266e-02, -1.4240e-02,  4.6427e-02,
         -1.3218e-01,  4.2161e-02,  2.6508e-02, -1.0130e-01, -1.1326e-02,
          3.0827e-03,  3.0376e-02, -4.8855e-02,  1.4727e-03, -3.6429e-02,
         -1.4679e-02, -5.5906e-02, -1.0308e-01,  9.6072e-03, -1.3606e-01,
          4.6313e-02,  9.3161e-02, -3.3545e-02,  2.9338e-02,  2.1417e-02,
          3.1531e-02, -2.3901e-02, -2.3531e-02,  7.3286e-02, -1.6327e-02,
         -9.8155e-02, -9.9542e-02, -5.7989e-02, -1.8593e-02,  1.1532e-02,
         -6.2275e-02, -6.3400e-02, -7.6797e-03, -2.9506e-02,  3.4842e-02,
         -5.0339e-02, -4.4179e-02, -5.1129e-03,  2.8546e-02,  2.0767e-02,
         -8.7036e-02, -2.3338e-02, -4.1307e-02, -4.8162e-02, -1.0762e-01,
         -5.1531e-02,  3.2062e-03, -6.1660e-02,  6.99

In [40]:
def parse_metadata(filename):
    pattern = r'\[(.*?)\] (\d{4}) - (.*?) \((.*?)(, (\d+)(ed))?, (\d{4})?\).pdf'
    match = re.match(pattern, filename)
    if match:
        type_, year, title, authors, _, edition, _, final_year = match.groups()
        return {
            'type': type_,
            'year': year,
            'title': title,
            'authors': authors,
            'edition': edition,
            'final_year': final_year
        }
    return None

def query_text(query, embeddings):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    results = []
    for filename, embedding in embeddings.items():
        similarity = util.pytorch_cos_sim(query_embedding, embedding)
        results.append((filename, similarity.item()))
    results.sort(key=lambda x: x[1], reverse=True)
    return results

In [41]:
def chatbot():
    print("Welcome to the PDF Library Manager Chatbot!")
    while True:
        user_input = input("Ask a question or type 'exit' to quit: ")
        if user_input.lower() == 'exit':
            break
        results = query_text(user_input, pdf_embeddings)
        if results:
            best_match = results[0]
            metadata = parse_metadata(best_match[0])
            if metadata:
                print(f"Best match found in: {metadata['title']} by {metadata['authors']} with similarity score of {best_match[1]:.4f}")
                print(f"Type: {metadata['type']}, Year: {metadata['year']}, Edition: {metadata['edition']}, Final Year: {metadata['final_year']}")
            else:
                print(f"Best match found in: {best_match[0]} with similarity score of {best_match[1]:.4f}")
        else:
            print("No relevant documents found.")

In [42]:
chatbot()

Welcome to the PDF Library Manager Chatbot!


NameError: name 'query_text' is not defined